In [0]:
#importing libraries
import tweepy
import os
import pandas as pd

In [0]:
#defining the keys
consumer_key='RhdMh9KGPCHSjPiUgJHFMOGE2'
consumer_secret='ehhQLcZjJYDBxPJnN72cZ6yn8sbyZZ8IUB3If7RcbXXJ8WTh31'
access_token='214710686-BNFZ4e6cd68FjLpLFSwtpl9s8vvJAIwpMuvQuCIf'
access_token_secret='BlyZSrV0GkXGchrfJdMEbETrDR1BkqeV1eK5ATN31Gw3P'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [0]:
def extract_coordinates(row):
    if row['Tweet Coordinates']:
        return row['Tweet Coordinates']['coordinates']
    else:
        return None

# Function created to extract place such as city, state or country from tweet if it has place info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None

In [0]:
#defining travel related keywords
text_keywords = ['travel','vacation','staycation','cruise']
coordinates = '1.352100,103.819800,50mi'
max_tweets = 150


tweet_df_list = []

for text_query in text_keywords:
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search, q = text_query, geocode = coordinates, count = 150).items(max_tweets)
    
    # List comprehension pulling chosen tweet information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.favorite_count, tweet.user.screen_name, 
                    tweet.user.id_str, tweet.user.location, tweet.user.followers_count, tweet.coordinates, tweet.place] for tweet in tweets]
    
    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Tweet Favorite Count', 'Twitter @ name',
                                                 'Twitter User Id', 'Twitter User Location', 'Users Current Following Count', 'Tweet Coordinates', 'Place Info'])
    
    
    # Checks if there are coordinates attached to tweets, if so extracts them
    tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
        
    # Checks if there is place information available, if so extracts them
    tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
    
    
    tweet_df_list.append(tweets_df)

In [0]:
#merging all the dataframes
tweet_df_all = pd.concat(tweet_df_list)

#resetting the index
tweet_df_all = tweet_df_all.reset_index(drop=True)

#validating
tweet_df_all.head()

Out[7]:

,Tweet Text,Tweet Datetime,Tweet Id,Tweet Favorite Count,Twitter @ name,Twitter User Id,Twitter User Location,Users Current Following Count,Tweet Coordinates,Place Info
0,S'pore and Malaysia to allow cross-border trav...,2021-05-02 08:34:04,1388773771926937603,0,HIREMAIDEA,1015456945447170048,Singapore,90,None,None
1,Alphard &amp; Vellfire Booking: Travel Singapo...,2021-05-02 08:33:32,1388773638959112193,0,Maxicabooking,922256694,Singapore,990,None,None
2,S'pore and Malaysia to allow cross-border trav...,2021-05-02 08:31:03,1388773013961666561,0,straits_times,37874853,Singapore,1085066,None,None
3,Singapore and Malaysia to allow cross-border t...,2021-05-02 08:29:32,1388772633303425028,0,sehof,10872822,Singapore,4630,None,None
4,New Zealand resumes Perth travel bubble after ...,2021-05-02 08:28:45,1388772434900230144,1,tocsg,44278933,Singapore,43043,None,None


In [0]:
#saving df to datalake
tweet_df_all.to_csv("/dbfs/mnt/agorizeblob/twitter_data/twitter_extract_sg.csv", index=False)